In [1]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import numpy as np
import os
import pyBigWig
from glob import glob
import pybedtools
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from scipy.stats import spearmanr

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [2]:
import os
import pandas as pd
import pyBigWig
from glob import glob
import pybedtools

# dirs for cage and feat datasets
CAGE_DIR = 'CAGE-train'
FEATURE_DIRS = {
    'DNase': 'DNase-bed',
    'DNase_bw': 'DNase-bigwig',
    'H3K4me1': 'H3K4me1-bed',
    'H3K4me1_bw': 'H3K4me1-bigwig',
    'H3K4me3': 'H3K4me3-bed',
    'H3K4me3_bw': 'H3K4me3-bigwig',
}

# chrom sets - trn, val, tst
SET_A = {'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr15', 'chr16', 'chr17', 'chr18', 'chr20', 'chr21', 'chr22'}
SET_B = {'chr14', 'chr19'}
SET_C = {'chr1'}

def load_gene_data(cell, split='train'):
    # load gene info/expr data
    info_path = os.path.join(CAGE_DIR, f"{cell}_{split}_info.tsv")
    info = pd.read_csv(info_path, sep='\t')
    
    # merge expr vals if not tst set
    if split != 'test':
        y_path = os.path.join(CAGE_DIR, f"{cell}_{split}_y.tsv")
        y = pd.read_csv(y_path, sep='\t')
        gene_data = pd.merge(info, y, on='gene_name')
    else:
        gene_data = info.copy()
    
    # tss based on strand info
    gene_data['TSS'] = gene_data.apply(lambda row: row['TSS_start'] if row['strand'] == '+' else row['TSS_end'] if row['strand'] == '-' else (row['TSS_start'] + row['TSS_end']) // 2, axis=1)
    return gene_data

def extract_bigwig_signal(bw_path, chrom, tss, window=1000):
    # get signal from bigwig file around tss
    try:
        with pyBigWig.open(bw_path) as bw:
            start, end = max(tss - window, 0), tss + window
            values = pd.Series(bw.values(chrom, start, end, numpy=True)).fillna(0)
            return values.mean()  
    except Exception as e:
        print(f"error bigwig {bw_path} for {chrom}:{start}-{end}: {e}")
        return 0.0

def add_bigwig_features(df, feature_name, bw_dir):
    # add bigwig feats each file in dir
    for bw_file in glob(os.path.join(bw_dir, '*.bw')) + glob(os.path.join(bw_dir, '*.bigwig')):
        cell = os.path.basename(bw_file).split('.')[0]  
        feature_col = f"{feature_name}_{cell}"
        print(f"extract bigwig feature {feature_col} from {bw_file}")
        df[feature_col] = df.apply(lambda row: extract_bigwig_signal(bw_file, row['chr'], row['TSS']), axis=1)
    return df

def count_bed_peaks(df, feature_name, bed_dir, window=1000):
    # bed peaks w/ window around tss
    for bed_file in glob(os.path.join(bed_dir, '*.bed')):
        cell = os.path.basename(bed_file).split('.')[0]
        feature_col = f"{feature_name}_{cell}_peak_count"
        print(f"count bed peaks {feature_col} from {bed_file}")
        
        # set window for peak count
        df['window_start'] = df['TSS'] - window
        df['window_end'] = df['TSS'] + window
        df['window_start'] = df['window_start'].apply(lambda x: max(x, 0))
        
        gene_windows = pybedtools.BedTool.from_dataframe(df[['chr', 'window_start', 'window_end']])
        peaks = pybedtools.BedTool(bed_file)
        
        intersections = gene_windows.intersect(peaks, wa=True, u=True)
        overlaps = intersections.to_dataframe(names=['chr', 'start', 'end'])
        
        # init peak count col
        df[feature_col] = 0
        for _, overlap in overlaps.iterrows():
            overlapping_genes = df[(df['chr'] == overlap['chr']) & (df['window_start'] <= overlap['start']) & (df['window_end'] >= overlap['end'])].index
            df.loc[overlapping_genes, feature_col] += 1

        df.drop(['window_start', 'window_end'], axis=1, inplace=True)
    
    return df

# load/prep train, val, test data
train_data = pd.concat([load_gene_data('X1', 'train'), load_gene_data('X2', 'train')], ignore_index=True)
train_data = train_data[train_data['chr'].isin(SET_A)]
print(f"train data: {train_data.shape[0]} genes")

val_data = pd.concat([load_gene_data('X1', 'val'), load_gene_data('X2', 'val')], ignore_index=True)
val_data = val_data[val_data['chr'].isin(SET_B)]
print(f"val data: {val_data.shape[0]} genes")

test_data = load_gene_data('X3', 'test')
test_data = test_data[test_data['chr'].isin(SET_C)]
print(f"test data: {test_data.shape[0]} genes")

# proccess bigwig feats
for feature, bw_dir in FEATURE_DIRS.items():
    if '_bw' in feature:
        train_data = add_bigwig_features(train_data, feature, bw_dir)
        val_data = add_bigwig_features(val_data, feature, bw_dir)
        test_data = add_bigwig_features(test_data, feature, bw_dir)

# bed peak counts
for feature, bed_dir in FEATURE_DIRS.items():
    if bed_dir.endswith('-bed'):
        train_data = count_bed_peaks(train_data, feature, bed_dir)
        val_data = count_bed_peaks(val_data, feature, bed_dir)
        test_data = count_bed_peaks(test_data, feature, bed_dir)

# save data
train_data.to_csv('train_prepared.csv', index=False)
val_data.to_csv('val_prepared.csv', index=False)
test_data.to_csv('test_prepared.csv', index=False)

print("data load/prep w/ bigwig & bed files complete.")


train data: 28620 genes
val data: 3948 genes
test data: 1984 genes
extract bigwig feature DNase_bw_X1 from DNase-bigwig/X1.bw
extract bigwig feature DNase_bw_X2 from DNase-bigwig/X2.bw
extract bigwig feature DNase_bw_X3 from DNase-bigwig/X3.bigwig
extract bigwig feature DNase_bw_X1 from DNase-bigwig/X1.bw
extract bigwig feature DNase_bw_X2 from DNase-bigwig/X2.bw
extract bigwig feature DNase_bw_X3 from DNase-bigwig/X3.bigwig
extract bigwig feature DNase_bw_X1 from DNase-bigwig/X1.bw
extract bigwig feature DNase_bw_X2 from DNase-bigwig/X2.bw
extract bigwig feature DNase_bw_X3 from DNase-bigwig/X3.bigwig
extract bigwig feature H3K4me1_bw_X2 from H3K4me1-bigwig/X2.bw
extract bigwig feature H3K4me1_bw_X3 from H3K4me1-bigwig/X3.bw
extract bigwig feature H3K4me1_bw_X1 from H3K4me1-bigwig/X1.bigwig
extract bigwig feature H3K4me1_bw_X2 from H3K4me1-bigwig/X2.bw
extract bigwig feature H3K4me1_bw_X3 from H3K4me1-bigwig/X3.bw
extract bigwig feature H3K4me1_bw_X1 from H3K4me1-bigwig/X1.bigwig
extr

## Work Package 1.2 - Model Building

In [4]:
# define spearman correlation scorer
def spearman_scorer(y_true, y_pred):
    rho, _ = spearmanr(y_true, y_pred)
    return rho

# scorer object for gridsearch
spearman_metric = make_scorer(spearman_scorer, greater_is_better=True)

# load prepped data
train_data = pd.read_csv('train_prepared.csv')
val_data = pd.read_csv('val_prepared.csv')

# feature cols and target var
excluded_columns = ['gene_name', 'chr', 'gene_start', 'gene_end', 
                    'TSS_start', 'TSS_end', 'strand', 'gex']

features = [col for col in train_data.columns if col not in excluded_columns]
target = 'gex'

# train/val sets
X_train = train_data[features]
y_train = train_data[target]
X_val = val_data[features]
y_val = val_data[target]

# xgboost regressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# hyperparam grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# gridsearchcv w/ spearman rho
grid_search = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid,
    scoring=spearman_metric,
    cv=5,
    n_jobs=-1,
    verbose=2
)

# fit model
print("starting grid search for xgboost...")
grid_search.fit(X_train, y_train)

# best model
best_xgb = grid_search.best_estimator_
print(f"best xgboost params: {grid_search.best_params_}")

# val set predictions
y_pred_val = best_xgb.predict(X_val)

# model eval
rho, _ = spearmanr(y_val, y_pred_val)  # spearman's rho
rmse = mean_squared_error(y_val, y_pred_val, squared=False)  # rmse

print(f"validation spearman rho: {rho:.4f}")
print(f"validation rmse: {rmse:.4f}")

# optional: save model
import joblib

model_filename = 'best_xgb_model.joblib'
joblib.dump(best_xgb, model_filename)
print(f"best xgboost model saved as '{model_filename}'.")

# optional: save val predictions
val_data['predicted_gex'] = y_pred_val
val_data.to_csv('val_predictions.csv', index=False)
print("validation preds saved to 'val_predictions.csv'.")


starting grid search for xgboost...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; 

## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [9]:
# TODO:
# Using the model trained in WP 1.2, make predictions on the test data (chr 1 of cell line X3).
# Store predictions in a variable called "pred" which is a numpy array.

pred = None
# ---------------------------INSERT CODE HERE---------------------------

best_xgb = joblib.load('best_xgb_model.joblib')
test_data = pd.read_csv('test_prepared.csv')

excluded_columns = ['gene_name', 'chr', 'gene_start', 'gene_end', 
                    'TSS_start', 'TSS_end', 'strand', 'gex']
features = [col for col in train_data.columns if col not in excluded_columns]

X_test = test_data[features]
pred = best_xgb.predict(X_test)
pred = np.array(pred)

test_genes = test_data['gene_name'].tolist()
#test_genes = pd.read_csv("CAGE-train/X3_test_info.tsv", sep='\t')

submission_df = pd.DataFrame({
    'gene_name': test_genes,
    'gex_predicted': pred
})
submission_df.index.name = ''
submission_df.to_csv('gex_predicted.csv', index=True)

print("Predictions on the test set have been made and stored in the 'pred' variable.")
print(type(pred))
print(pred.shape)

# ----------------------------------------------------------------------

#test_genes = test_data['gene_name'].tolist()
test_genes = pd.read_csv("CAGE-train/X3_test_info.tsv", sep='\t')

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

Predictions on the test set have been made and stored in the 'pred' variable.
<class 'numpy.ndarray'>
(1984,)


#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = '' 
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "Stebler_Yves_Project1.zip" 
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes['gex_predicted'] = pred.tolist()
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)

In this project, we developed a machine learning model to predict gene expresion levels from chromatin features. We began by preprocesing histone modification and chromatin accesibility data, focusing on specific genomic regions around the transcription start site (TSS) to capture meaningful signals. Chromosome data from cell lines X1 and X2 was used for training on chr2-22, then validated on chr14 and chr19. An XGBoost regressor model was trained, and predictions were made on chr1 from cell line X3. Spearman’s correlation served as a key metric, showing strong alignment with actual expresion. While we did not use the full dataset we still achieved enough performance to pass the baseline which was the main goal. 